In [13]:
"""
Q37. 플로이드
https://acmicpc.net/problem/11404 
시간: 20:05~20:35
모든 도시의 쌍 (A, B)에 대해서 도시 A에서 B로 가는데 필요한 비용의 최솟값이므로 플로이드-워셜 알고리즘을 1번 돌리면 구할 수 있다.

시작 도시와 도착 도시를 연결하는 노선은 하나가 아닐 수 있다 -> 플로이드 탐색 시작 전에 입력값으로 주어진 비용을 graph에 추가할 때 이전 값보다
작은지 검증해주는 절차가 필요하다. 

시간복잡도가 O(n^3)이라서 그런지 실행 속도가 엄청 느리다. 
input()대신 sys.stdin.readline()을 써야함(기존 답지 4328ms에서 632ms로 줄임)
"""
from math import inf
import sys

cities = int(input())
bus = int(input())

# 2차원 그래프 생성. 초기값은 INF, 자기 자신과의 거리는 0으로 초기화
graph = [[inf for _ in range(cities + 1)] for _ in range(cities + 1)]
for i in range(1, cities + 1):
    for j in range(1, cities + 1):
        if i == j:
            graph[i][j] = 0

# 버스 노선 정보를 받아서 graph에 반영 (기존 비용보다 작은 지 검증)
for _ in range(bus):
    i, j, value = map(int, sys.stdin.readline().split())
    graph[i][j] = value if value < graph[i][j] else graph[i][j]


# 3중 for문으로 모든 노드 간 간선의 최소비용을 체크
for k in range(1, cities+1):
    for a in range(1, cities+1):
        for b in range(1, cities+1):
            # 모든 노드들에 대해 현재 노드를 거쳐서 가는 모든 N^2 개의 경로를 고려함
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])

# 간선 비용이 inf인 경우를 제외하고 간선 비용들을 각 줄에 하나씩 출력
for i in range(1, cities+1):
    for j in range(1, cities+1):
        if graph[i][j] == inf:
            print(0, end=" ")
        else:
            print(graph[i][j], end=" ")
    print()

[inf, inf, inf, inf, inf, inf]
[inf, 0, 2, 3, 1, 10]
[inf, inf, 0, inf, 2, inf]
[inf, 8, inf, 0, 1, 1]
[inf, inf, inf, inf, 0, 3]
[inf, 7, 4, inf, inf, 0]
0 2 3 1 4 
12 0 15 2 5 
8 5 0 1 1 
10 7 13 0 3 
7 4 10 6 0 


In [29]:
"""
Q38. 정확한 순위 

학생 수가 n이라고 했을 때, 임의의 학생의 순위를 알 수 있는 조건은, 
학생들 간의 상대적인 등수 비교를 2차원 adjancency adj_list로 나타냈을 때 해당 학생이 다른 학생들과 연결된 간선이 자기 자신을 포함해 n-1개면 알 수 있다. 
ex. 총 6명이 있다고 했을 때, 내 위로 2명, 내 아래로 3명이 있다는 정보가 있으면 나는 3등이라는 걸 알 수 있음. 2+3 = 6-1

입력값으로 주어지는 순위 비교만으로는 부족하고, 
if 1 < 2 and 2 < 4, then 1 < 4 처럼 논리적 유추를 통해서 주어지는 간선 외에도 논리적 추론이 가능한 간선들을 전부 다 탐색해야 함.
쉬운 탐색을 위해 adj_matrix외에도 동일한 정보를 담은 adj list를 따로 생성함. 
"""
from math import inf

students, comparisons = map(int, input().split())

# adj matrix 생성
graph = [[inf] * (comparisons + 1) for _ in range(comparisons + 1)]
for i in range(1, comparisons + 1):
    graph[i][i] = 0 # 자기 자신과의 관계는 0으로 책정

# adj list 생성
adj_list = [[] for _ in range(comparisons + 1)]
for i in range(comparisons):
    a, b = map(int, input().split())
    adj_list[a].append((a, b))  

# queue 생성
queue = set()

# 탐색 시작
for i in range(1, comparisons + 1):
    for j in adj_list[i]:
        queue.add(j)

    while queue:
        a, b = queue.pop() # 성적 비교 결과 가져오기
        
        if graph[a][b] == inf:  # 미방문일 경우에는
            graph[a][b] = 1 # 비교 결과 1 저장
            graph[b][a] = 1 # 상대 관계의 간선은 양방향으로 존재하므로 반대 시점에서의 관계도 저장

            for p in adj_list[b]: # 노드 b에 연결되어 있는 다른 간선들을 모두 queue에 삽입  # adj_list[b] = [(4, 2), (4, 6)]
                queue.add((a, p[1]))

# 결과 출력
# 특정 row의 모든 element를 합했을 때 만약 상대 등수가 불명확한 학생이 존재한다면 값이 갱신되지 않고 inf로 남아있을 것이므로, 
# row의 합이 inf가 아닌 학생의 수를 반환하면 된다. 
result = 0
for i in range(1, students + 1):
    sum = 0
    for j in range(1, students + 1):
        sum += graph[i][j]
    if sum < inf:
        result += 1
print(result)
            

1


In [30]:
for i in graph:
    print(i)

[inf, inf, inf, inf, inf, inf, inf]
[inf, 0, 1, inf, 1, 1, 1]
[inf, 1, 0, 1, 1, 1, inf]
[inf, inf, 1, 0, 1, inf, 1]
[inf, 1, 1, 1, 0, 1, 1]
[inf, 1, 1, inf, 1, 0, 1]
[inf, 1, inf, 1, 1, 1, 0]


In [ ]:
"""
Q39. 화성 탐사
"""

In [ ]:
"""
Q40. 숨바꼭질
"""